In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx

import matplotlib.pyplot as plt

In [2]:
!python3 -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 6.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\T-GAMER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
#carregando modelo
model = 'en_core_web_sm'
NER = spacy.load(model)


In [10]:

book_text= open('../Book/lightning_thief_the__percy_jac_-_rick_riordan.txt', encoding='ISO-8859-1').read()
NER.max_length = 2000000
book = NER(book_text)

In [11]:
#Verificando se ta tudo certo
displacy.render(book[108700:109000], style="ent", jupyter=True)

In [13]:
sentence= []

for i in book.sents:
    ent_list=(ent.text for ent in i.ents)
    sentence.append({"sentence":i,'entity':ent_list})

In [14]:
sentence=pd.DataFrame(sentence)
sentence.head()

,sentence,entity
0,"( , BOOKS, BY, RICK, RIORDAN, \n\n\n ...",<generator object <genexpr> at 0x00000221C1C15...
1,"(The, Demigod, Files, \n\n , Percy, Jacksonâ...",<generator object <genexpr> at 0x00000221B82DF...
2,"(The, Graphic, Novel, \n\n , The, Sea, ...",<generator object <genexpr> at 0x00000221C1C15...
3,"(The, Red, Pyramid, \n , The,...",<generator object <genexpr> at 0x00000221C1C15...
4,"(The, Red, Pyramid, :, The, Graphic, Novel, \n...",<generator object <genexpr> at 0x00000221C1C15...


In [16]:
character_df=pd.read_excel('../Data/Character.xlsx')

In [17]:
character_df["first_name"] = character_df["Character"].apply( lambda x: x.split(" ",1)[0])

In [19]:

def filter_out(entity_col, character_df):
    return [x for x in entity_col if x in list(character_df['first_name'])]


sentence['character_entity'] = sentence['entity'].apply(lambda x: filter_out(x, character_df))

In [21]:
df_filter = sentence[sentence['character_entity'].apply(lambda x: len(x) > 0)].reset_index()

In [22]:
df_filter.head()

,index,sentence,entity,character_entity
0,73,"(All, the, way, into, the, city, ,, I, put, up...",<generator object <genexpr> at 0x00000221BC120...,[Grover]
1,85,"(Grover, tried, to, calm, me, down, .)",<generator object <genexpr> at 0x00000221BC102...,[Grover]
2,88,"(âThatâs, it.â, I, started, to, get, up,...",<generator object <genexpr> at 0x00000221BA3BF...,[Grover]
3,103,"(One, time, ,, after, sheâd, made, me, erase...",<generator object <genexpr> at 0x00000221BA3BE...,[Grover]
4,113,"(âThatâs, Kronos, eating, his, kids, ,, ri...",<generator object <genexpr> at 0x00000221BC121...,[Kronos]


In [23]:

window_size = 5
char_list = []
for window in df_filter['character_entity'].rolling(window=window_size):
    if window.size == window_size:
        char_list.extend(sum(window.dropna(), [])) 

In [24]:

unique_list = []

prev_element = None


for element in char_list:
    if element != prev_element:
        unique_list.append(element)
 
    prev_element = element

In [25]:
unique_list[:10]

['Grover',
 'Kronos',
 'Grover',
 'Kronos',
 'Grover',
 'Kronos',
 'Zeus',
 'Kronos',
 'Grover',
 'Kronos']

In [26]:

source = []
target = []

for i in range(len(unique_list)-1):
    source.append(unique_list[i])
    target.append(unique_list[i+1])       

relationship_df = pd.DataFrame({"Source" : source,
                               "Target" : target})

In [28]:
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df

,Source,Target
0,Grover,Kronos
1,Grover,Kronos
2,Grover,Kronos
3,Grover,Kronos
4,Grover,Kronos
...,...,...
4677,Annabeth,Tyson
4678,Annabeth,Tyson
4679,Annabeth,Tyson
4680,Annabeth,Tyson


In [29]:

relationship_df["Weight"] = 1
relationship_df = relationship_df.groupby(["Source","Target"], sort=False, as_index=False).sum()

In [30]:
relationship_df=relationship_df.sort_values(by='Weight',ascending=False)


In [31]:
relationship_df

,Source,Target,Weight
12,Annabeth,Grover,630
3,Grover,Percy,210
17,Chiron,Percy,197
5,Chiron,Grover,195
15,Annabeth,Percy,184
...,...,...,...
113,Furies,Percy,1
137,Cerberus,Percy,1
62,Hades,Thalia,1
43,Clarisse,Grover,1


In [32]:
relationship_df.to_csv("../Data/relationship_percy", index=False) 